In [111]:
# ===========================================> Carga librerías <====================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score, confusion_matrix, auc, plot_roc_curve, roc_curve, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, auc, RocCurveDisplay , roc_curve, classification_report

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [112]:
# ===========================================> Carga Datos <========================================================

datos = pd.read_csv('../../data/raw/test.csv', low_memory=False)

In [113]:
# ===========================================> Carga Procesados Train <=============================================

cat_cols = pickle.load(open("../../columns/cat.pkl", 'rb'))
cat_cols_mask_3 = pickle.load(open("../../columns/cat_mask_3.pkl", 'rb'))
cat_cols_mask_4 = pickle.load(open("../../columns/cat_mask_4.pkl", 'rb'))
cat_cols_mask_5 = pickle.load(open("../../columns/cat_mask_5.pkl", 'rb'))
cat_cols_mask_6 = pickle.load(open("../../columns/cat_mask_6.pkl", 'rb'))
cat_cols_target_encoder = pickle.load(open("../../columns/cat_target_encoder.pkl", 'rb'))
cat_cols_onehot_encoder = pickle.load(open("../../columns/cat_onehot_encoder.pkl", 'rb'))
cat_cols_pro = pickle.load(open("../../columns/cat_pro.pkl", 'rb'))

num_cols = pickle.load(open("../../columns/num.pkl", 'rb'))
num_cols_onehot_encoder = pickle.load(open("../../columns/num_onehot_encoder.pkl", 'rb'))
num_cols_correlacion = pickle.load(open("../../columns/num_correlacion.pkl", 'rb'))
num_cols_pro = pickle.load(open("../../columns/num_pro.pkl", 'rb'))

target_encoder = pickle.load(open("../../encoders/target.pkl", 'rb'))
onehot_encoder = pickle.load(open("../../encoders/onehot.pkl", 'rb'))

intercuartilico = pickle.load(open("../../interquartiles/intercuartilico.pkl", 'rb'))

cat_simple_imputer = pickle.load(open("../../imputers/cat_simple.pkl", 'rb'))
num_simple_imputer = pickle.load(open("../../imputers/num_simple.pkl", 'rb'))

In [114]:
# ===========================================> Definir variables <==================================================

IDENTIFIER = "MachineIdentifier"
LABEL = "HasDetections"

In [115]:
# ===========================================> Eliminar columnas nuevas <===========================================

datos = datos[[IDENTIFIER]+cat_cols+num_cols]

In [116]:
# ===========================================> Separar datos por tipos categoricas <================================

datos[cat_cols] = datos[cat_cols].astype("category")

In [117]:
# ===========================================> Separar datos por tipos numericas <==================================

datos[num_cols] = datos[num_cols].astype("float64")

In [118]:
# ===========================================> Imputar nulos para tipos categoricas <===============================

datos[cat_cols] = cat_simple_imputer.transform(datos[cat_cols])

In [119]:
# ===========================================> Procesar mascaras y versiones 3 partes <=============================

for col in cat_cols_mask_3:
    datos[[col + "_1", col + "_2", col + "_3"]] = datos[col].str.split(".", expand=True)

for col in cat_cols_mask_3:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")

datos.drop(columns=cat_cols_mask_3, inplace=True)

In [120]:
# ===========================================> Procesar mascaras y versiones 4 partes <=============================

for col in cat_cols_mask_4:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4"]] = datos[col].str.split(".", expand=True)

for col in cat_cols_mask_4:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")    

datos.drop(columns=cat_cols_mask_4, inplace=True)

In [121]:
# ===========================================> Procesar mascaras y versiones 5 partes <=============================

for col in cat_cols_mask_5:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5"]] = datos[col].str.split(".", expand=True)

for col in cat_cols_mask_5:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")

datos.drop(columns=cat_cols_mask_5, inplace=True)

In [122]:
# ===========================================> Procesar mascaras y versiones 6 partes <=============================

for col in cat_cols_mask_6:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5", col + "_6"]] = datos[col].str.split(".", expand=True)

for col in cat_cols_mask_6:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")
    cat_cols.append(col + "_6")

datos.drop(columns=cat_cols_mask_6, inplace=True)

In [123]:
# ===========================================> Procesar target encoder <============================================

datos[cat_cols_target_encoder] = target_encoder.transform(datos[cat_cols_target_encoder])

for col in cat_cols_target_encoder:
    cat_cols.remove(col)
    num_cols.append(col)

In [124]:
# ===========================================> Procesar onehot encoder <============================================

onehot_datos = onehot_encoder.transform(datos[cat_cols_onehot_encoder]).toarray()
num_cols_onehot_encoder_test = onehot_encoder.get_feature_names_out(cat_cols_onehot_encoder)

datos[num_cols_onehot_encoder_test] = pd.DataFrame(onehot_datos, columns=num_cols_onehot_encoder_test)[num_cols_onehot_encoder_test]

datos.drop(columns=cat_cols_onehot_encoder, inplace=True)

for col in cat_cols_onehot_encoder:
    cat_cols.remove(col)

for col in num_cols_onehot_encoder_test:
    num_cols.append(col)  

C:\Users\jcosta\AppData\Local\Temp\ipykernel_30924\4241278943.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datos[num_cols_onehot_encoder_test] = pd.DataFrame(onehot_datos, columns=num_cols_onehot_encoder_test)[num_cols_onehot_encoder_test]
C:\Users\jcosta\AppData\Local\Temp\ipykernel_30924\4241278943.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datos[num_cols_onehot_encoder_test] = pd.DataFrame(onehot_datos, columns=num_cols_onehot_encoder_test)[num_cols_onehot_encoder_test]
C:\Users\jcosta\AppData\Local\Temp\ipyke

In [125]:
# ===========================================> Imputar nulos para tipos numericas <=================================

datos[num_cols] = num_simple_imputer.transform(datos[num_cols])

In [126]:
# ===========================================> Tratamiento de valores atípicos <====================================

#for col in num_cols:
#    datos[col] = datos[col].apply(lambda x: x if intercuartilico[col]["lower"] <= x <= intercuartilico[col]["upper"] else intercuartilico[col]["mean"])

#for col in num_cols:
#    datos[col] = datos[col].apply(lambda x: None if x < intercuartilico[col]["lower"] or x > intercuartilico[col]["upper"] else x)

#datos[num_cols] = datos[num_cols][~((datos[num_cols] < (intercuartilico["Q1"] - 1.5 * intercuartilico["IQR"])) |(datos[num_cols] > (intercuartilico["Q3"] + 1.5 * intercuartilico["IQR"]))).any(axis=1)]
#datos[num_cols] = datos[num_cols][~((datos[num_cols] < (intercuartilico["Q1"] - 1.5 * intercuartilico["IQR"])) |(datos[num_cols] > (intercuartilico["Q3"] + 1.5 * intercuartilico["IQR"])))].any(axis=1)

In [127]:
# ===========================================> Imputar nulos para tipos numericas <=================================

#num_simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

#num_simple_imputer = num_simple_imputer.fit(datos[num_cols])
#datos[num_cols] = num_simple_imputer.transform(datos[num_cols])

#datos[num_cols].isnull().sum()

In [128]:
# ===========================================> Eliminar por Matriz de correlación <=================================

datos.drop(columns=num_cols_correlacion, inplace=True)

for col in num_cols_correlacion:
    num_cols.remove(col)

In [129]:
# ===========================================> Reiniciar Separar datos por tipos categoricas <======================

cat_cols = cat_cols_pro

datos[cat_cols] = datos[cat_cols].astype("category")

In [130]:
# ===========================================> Reiniciar Separar datos por tipos numericas <========================

num_cols = num_cols_pro

datos[num_cols] = datos[num_cols].astype("float64")

In [131]:
# ===========================================> Ordenar variables <==================================================

datos = datos[[IDENTIFIER]+cat_cols+num_cols]

In [132]:
# ===========================================> Guardamos los datos preprocesados <==================================

datos.to_csv('../../data/processed/test.csv', index=False)